In [114]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import gensim.downloader as api
import pandas as pd

In [115]:
wv = api.load('glove-twitter-25')

In [116]:
data = pd.read_csv('/Users/tanhoangminhco/Downloads/Downloads/reddit_worldnews_start_to_2016-11-22.csv')
data = data['title']

In [117]:
tweets = data[:10]
max_in = 16
n_dim = 25
total_tokens = 1193514

In [118]:
def count(tweets):
    max = 0
    for tweet in tweets:
        if len(tweet.split()) > max:
            max =  len(tweet.split())
    return max

In [119]:
def embed(tokens, max_input):
    return embedRecur(tokens, index=0, max_input=max_input, array=[])

def embedRecur(tokens, index, max_input, array):
    if index==len(tokens):
        if index < max_input:
            for i in range(index, max_input):
                array.append(np.zeros(n_dim))
        return np.array(array)
    try:
        emb = wv[tokens[index]]
    except KeyError:
        array.append(wv['unk'])
    else:
        array.append(emb)

    return embedRecur(tokens, index + 1, max_input, array)

In [120]:
input1 = []
input2 = []
output = []
for tweet in tweets:
    tokens = tweet.lower().split()
    for i in range(1, len(tokens)-1):
        in1_tokens = tokens[:i]
        in1 = embed(in1_tokens, max_in)
        in1 = in1.reshape(1,max_in,n_dim)

        for y in range(i, len(tokens)):
            input1.append(in1)
            in2_tokens = ['start'] + tokens[i: y]
            in2 = embed(in2_tokens, max_in)
            input2.append(in2.reshape(1,max_in,n_dim))
            try:
                index = wv.key_to_index[tokens[y]]
            except KeyError:
                index = wv.key_to_index['unk']
            
            # out = tf.keras.utils.to_categorical(index, num_classes=total_tokens, dtype='float16').reshape(1,1,-1)
            output.append(index)


input1 = np.array(input1)
input2 = np.array(input2)
output = np.array(output)

In [121]:
input1_to_file = input1.reshape(-1, n_dim)
np.savetxt('/Users/tanhoangminhco/Documents/Coding/Python/Machine Learning/Deep Learning/DLmodels/transformer/data/input1.csv',
           input1_to_file)

In [122]:
input2_to_file = input2.reshape(-1, n_dim)
np.savetxt('/Users/tanhoangminhco/Documents/Coding/Python/Machine Learning/Deep Learning/DLmodels/transformer/data/input2.csv',
           input2_to_file)

In [123]:
output_to_file = output

np.savetxt('/Users/tanhoangminhco/Documents/Coding/Python/Machine Learning/Deep Learning/DLmodels/transformer/data/output.csv',
           output_to_file)